# Fluid dynamics · Lid-driven cavity walkthrough

A lightweight, coarse-grid reproduction of `code/fluid_dynamics/lid_cavity_benchmark.py`. We run a short LBM simulation, track the peak velocity, and note where the full benchmark would archive its artifacts.


In [ ]:
import sys
from pathlib import Path

def _ensure_repo_on_path():
    here = Path.cwd().resolve()
    for candidate in (here, *list(here.parents)[:4]):
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            path_str = str(candidate)
            if path_str not in sys.path:
                sys.path.insert(0, path_str)
            return candidate
    return here

_ensure_repo_on_path()


In [ ]:
from notebooks.utils import ensure_repo_path, allocate_artifacts, preview_json, summarize_array

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")


In [ ]:
try:
    from code.fluid_dynamics.fluids.lbm2d import LBMConfig, LBM2D
    from code.fluid_dynamics.lid_cavity_benchmark import lbm_viscosity_from_tau
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from code.fluid_dynamics.fluids.lbm2d import LBMConfig, LBM2D
    from code.fluid_dynamics.lid_cavity_benchmark import lbm_viscosity_from_tau

import numpy as np

cfg = LBMConfig(
    nx=48,
    ny=48,
    tau=0.75,
    periodic_x=False,
    periodic_y=False,
    void_enabled=False,
    u_clamp=0.08,
)
sim = LBM2D(cfg)
sim.set_solid_box(top=False, bottom=True, left=True, right=True)
U_lid = 0.04

for step in range(80):
    sim.step(1)
    sim.set_lid_velocity(U_lid)
    if step % 20 == 0:
        sim.moments()
        speed = np.sqrt(sim.ux**2 + sim.uy**2)
        print(f"step {step:02d}: max|u|={float(np.nanmax(speed)):.4f}")

sim.moments()
speed_stats = summarize_array(np.sqrt(sim.ux**2 + sim.uy**2))
print(preview_json({
    "nu": lbm_viscosity_from_tau(cfg.tau),
    "max_speed": speed_stats.get("max"),
    "mean_speed": speed_stats.get("mean"),
}))


In [ ]:
artifacts = allocate_artifacts("fluid_dynamics", "lid_cavity_notebook_demo")
print(preview_json({name: str(path) for name, path in artifacts.items()}))


## Notebook checklist

* The coarse grid (`48×48`, 80 steps) keeps turnaround sub-second; raise the resolution for production plots.
* Cavity walls are enforced via bounce-back (`set_solid_box`) while the lid uses the Zou/He velocity rule.
* `summarize_array` offers a quick health-check on the velocity field without plotting.
* Saved figures/logs reuse the benchmark’s routing, so CLI runs and notebooks share the same artifact tree.
